In [1]:
!pip install transformers==4.34.1
!pip install mlflow torchvision einops


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import transformers
import mlflow
import os
import logging
from transformers import AutoModelForCausalLM, AutoTokenizer
MLFLOW_ROUTE = os.getenv("MLFLOW_ROUTE")
print(MLFLOW_ROUTE)


http://mlflow-server.mlflow.svc.cluster.local:8080


In [3]:
mlflow.set_tracking_uri(MLFLOW_ROUTE)
mlflow.set_experiment("yi34b")

<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1700576882759, experiment_id='3', last_update_time=1700576882759, lifecycle_stage='active', name='yi34b', tags={}>

In [ ]:
logging.getLogger("mlflow").setLevel(logging.DEBUG)
tokenizer = AutoTokenizer.from_pretrained("01-ai/Yi-34B", trust_remote_code=True)

chat_pipeline = transformers.pipeline(model="01-ai/Yi-34B", trust_remote_code=True, tokenizer=tokenizer)

In [ ]:
with mlflow.start_run():
  model_info = mlflow.transformers.log_model(
    transformers_model=chat_pipeline,
    artifact_path="yi34bmodel",
    input_example="hi there"
  )


In [4]:
# Load as interactive pyfunc

chatbot = mlflow.pyfunc.load_model("mlflow-artifacts:/2/5bbf49af4e714aeb9b6be13f9c9ee53c/artifacts/phi15model")
#make predictions
#chatbot.predict("What is the best way to get to Antarctica?")

chatbot.predict("Which kubernetes should should I use? Red Hat or VMWare?")


2023/11/22 00:43:43 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


MlflowException: The following failures occurred while downloading one or more artifacts from http://mlflow-server.mlflow.svc.cluster.local:8080/api/2.0/mlflow-artifacts/artifacts/2/5bbf49af4e714aeb9b6be13f9c9ee53c/artifacts:
##### File phi15model/model/pytorch_model-00004-of-00012.bin #####
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
##### File phi15model/model/pytorch_model-00005-of-00012.bin #####
('Connection broken: IncompleteRead(1058 bytes read, 50 more expected)', IncompleteRead(1058 bytes read, 50 more expected))
##### File phi15model/model/pytorch_model-00010-of-00012.bin #####
("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
##### File phi15model/model/pytorch_model-00007-of-00012.bin #####
('Connection broken: IncompleteRead(227 bytes read, 757 more expected)', IncompleteRead(227 bytes read, 757 more expected))
##### File phi

In [16]:
mlflow.artifacts.download_artifacts(artifact_uri="mlflow-artifacts:/2/5bbf49af4e714aeb9b6be13f9c9ee53c/artifacts/phi15model" ,dst_path='servepath')

2023/11/21 14:19:39 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


MlflowException: The following failures occurred while downloading one or more artifacts from http://mlflow-server.mlflow.svc.cluster.local:8080/api/2.0/mlflow-artifacts/artifacts/2/5bbf49af4e714aeb9b6be13f9c9ee53c/artifacts:
##### File phi15model/model/pytorch_model-00007-of-00012.bin #####
('Connection broken: IncompleteRead(0 bytes read, 5 more expected)', IncompleteRead(0 bytes read, 5 more expected))
##### File phi15model/model/pytorch_model-00004-of-00012.bin #####
('Connection broken: IncompleteRead(712 bytes read, 892 more expected)', IncompleteRead(712 bytes read, 892 more expected))
##### File phi15model/model/pytorch_model-00003-of-00012.bin #####
('Connection broken: IncompleteRead(698 bytes read, 1278 more expected)', IncompleteRead(698 bytes read, 1278 more expected))

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("./servepath/chatbot/components/tokenizer")
model = AutoModelForCausalLM.from_pretrained("./servepath/chatbot/model")

# Let's chat for 5 lines
for step in range(1):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

>> User: D


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: D


In [18]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

#torch.set_default_device("cuda")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
inputs = tokenizer('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx